<h1><center> Foursquare Location Matching </center></h1>
<h3><center> Model Training </center></h3>
<h3><center> Tao Shan </center></h3>

This notebook train model and save the trained model for further uses. You can change the model to other models to have a try, such as Xgboost, random forest, Logistic regression, NLP.

### Other Relevant notebooks and links

Competition: [Foursquare - Location Matching](https://www.kaggle.com/competitions/foursquare-location-matching)

Train data generation notebook: [Foursquare - train data generation](https://www.kaggle.com/taos2000/foursquare-train-data-generation)

Train data preprocessing notebook: [Foursquare - train data preprocess](https://www.kaggle.com/taos2000/foursquare-train-data-preprocess)

Model Selection: [Foursquare - model selection](https://www.kaggle.com/taos2000/foursquare-model-selection)

Model Training: [Foursquare - model training](https://www.kaggle.com/taos2000/foursquare-model-training)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from pickle import dump
import gc
import time
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from geopy.geocoders import Nominatim

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer

import Levenshtein as lev
import math
from collections import Counter

from pickle import dump, load
import time
from sklearn.neighbors import BallTree

import itertools
from tqdm.auto import tqdm
tqdm.pandas()
import gc

from fuzzywuzzy import fuzz
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [2]:
train_pairs = pd.read_pickle('../input/last-day-four-points/train_pairs (1).pkl')

In [3]:
cols = ['address_1_missing',
        'country_same',
 'address_2_missing',
 'categories_1_count',
 'categories_1_count_diff',
 'categories_2_count',
 'city_1_count',
 'city_1_count_diff',
 'city_1_missing',
 'city_2_count',
 'city_2_missing',
 'country_1_count',
 'country_1_count_diff',
 'distance',
 'latitude_1',
 'latitude_1_count',
 'latitude_2_count',
 'latitude_2_count_diff',
 'longitude_1',
 'longitude_1_count',
 'longitude_2_count',
 'longitude_2_count_diff',
 'phone_1_missing',
 'phone_2_missing',
 'state_1_count',
 'state_1_count_diff',
 'state_2_count',
 'url_1_missing',
 'url_2_missing',
 'zip_1_missing',
 'zip_2_missing',
 'name_1_fuzzy',
 'name_1_fuzzy_partial',
 'address_1_fuzzy',
 'categories_1_fuzzy',
 'categories_1_fuzzy_partial',
 'url_1_fuzzy']
ids = ['id_1','id_2']
cols_new = ['address_1_missing',
 'address_2_missing',
 'city_1_missing',
 'city_2_missing',
 'distance',
 'latitude_1',
 'longitude_1',
 'phone_1_missing',
 'phone_2_missing',
 'url_1_missing',
 'url_2_missing',
 'zip_1_missing',
 'zip_2_missing',
 'name_1_fuzzy',
 'name_1_fuzzy_partial',
 'address_1_fuzzy',
 'categories_1_fuzzy',
 'categories_1_fuzzy_partial',
 'url_1_fuzzy']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(train_pairs[cols_new], train_pairs['match'], test_size=0.1, random_state=42)

In [5]:
scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
xgb = CatBoostClassifier(eval_metric = 'Accuracy', depth = 5,verbose=False)
xgb.fit(X_train_temp, y_train,
       eval_set=(X_test_temp, y_test))

In [6]:
xgb.save_model("model.cbm")

model.load_model("model.cbm")

scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
gc.collect()
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
                  importance_type='gain', objective = 'binary:logistic',
                  learning_rate=0.1, max_delta_step=0, max_depth=3,
                  min_child_weight=2, n_estimators=400, n_jobs=-1, nthread=-1, num_parallel_tree=1,
                  random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                  subsample=0.9, tree_method='exact', validate_parameters=1,eval_metric = 'auc',
                  verbosity=0)
gc.collect()
xgb.fit(X_train_temp, y_train, eval_set=[(X_train_temp, y_train),(X_test_temp, y_test)])

In [7]:
#xgb.save_model("model.json")

In [8]:
dump(scaler, open('scaler.pkl', 'wb')) # save the scaler